# Imports

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container{max-width:80%!important;width:auto!important;}</style>"))

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import trange # https://tqdm.github.io/

In [ ]:
def show_img(im, ax=None, figsize=(8,8)):
    if not ax: _,ax = plt.subplots(1,1,figsize=figsize)
    if len(im.shape)==2: im = np.tile(im[:,:,None], 3) 
    ax.imshow(im[:,:,::-1]);
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    return ax

# Image feature descriptors

## Corner detection

### Harris Corner Detection

In [ ]:
im = cv2.imread('Data/BUlogor.jpg')
show_img(im, figsize=(16,16));

In [ ]:
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY).astype(np.float32)

In [ ]:
??cv2.cornerHarris

In [ ]:
dst = cv2.cornerHarris(gray, 2, 3, 0.04)
show_img(cv2.convertScaleAbs(dst), figsize=(16,16));

In [ ]:
im_copy = im.copy()
for pt in np.argwhere(dst>0.05*dst.max()):
    im_copy = cv2.circle(im_copy, tuple(pt[::-1]), 5, (0,255,0), 10)
show_img(im_copy, figsize=(12,12));

In [ ]:
plt.hist(dst.flatten(), 50, log=True);

### Shi-Tomasi Corner Detection

In [ ]:
im_copy = im.copy()
corners = cv2.goodFeaturesToTrack(gray, 70, 0.01, 5, useHarrisDetector=False)
for c in corners:
    im_copy = cv2.circle(im_copy, tuple(c.squeeze()), 5, (0,255,0), 10)
show_img(im_copy, figsize=(12,12));

## SIFT and SURF

In [ ]:
im = cv2.imread('05-files/fusion-building.jpg')
show_img(im, figsize=(16,16));

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()
kp, des = sift.detectAndCompute(im, None)

In [ ]:
im_copy = im.copy()
cv2.drawKeypoints(im, kp, im_copy, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
show_img(im_copy, figsize=(16,16));

In [ ]:
des.shape

In [ ]:
surf = cv2.xfeatures2d.SURF_create(1329)
kp, des = surf.detectAndCompute(im, None)
des.shape

In [ ]:
im_copy = im.copy()
cv2.drawKeypoints(im, kp, im_copy, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
show_img(im_copy, figsize=(16,16));

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()
surf = cv2.xfeatures2d.SURF_create(1329, upright=False)
surf.setExtended(True)

## FAST + BRIEF = ORB

In [ ]:
im = cv2.imread('Data/BUlogor.jpg')
orb = cv2.ORB_create()
kp = orb.detect(im, None)
kp, des = orb.compute(im, kp)
img2 = cv2.drawKeypoints(im, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
show_img(img2, figsize=(16,16));

In [ ]:
orb = cv2.ORB_create()
%timeit -n10 orb.compute(im, orb.detect(im, None))

# Feature matching

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
def show_imgs(ims, rows=1, figsize=(16,16)):
    _,ax = plt.subplots(rows, len(ims)//rows, figsize=figsize)
    [show_imgs(im,ax_) for im,ax_ in zip(ims,ax.flatten())]
    return ax

In [ ]:
im1 = cv2.imread('Data/trial.jpeg')

im2 = cv2.imread('Data/BUlogor.jpg')
show_imgs((im1,im2), rows=1, figsize=(16,16));

## Brute-force matcher

The [brute-force matcher](https://docs.opencv.org/3.4.9/dc/dc3/tutorial_py_matcher.html) tries to match each and every feature in the first image with each and every features in the second image, which is slow. If the first image has $n_1$ features and the second image has $n_2$ features, the brute-force approach will evaluate $n_1 \times n_2$ potential matches.

In [ ]:
descr = cv2.xfeatures2d.SURF_create()
# descr = cv2.ORB_create()
kp1,des1 = descr.detectAndCompute(im1,None)
kp2,des2 = descr.detectAndCompute(im2,None)

`cv2.BFMatcher` takes two arguments:
- `normType` - denotes *how* to calculate the similarity of the descriptors; `cv2.NORM_L2` (i.e. Euclidean) is the default, which works well for SIFT and SURF; for ORB `cv2.NORM_HAMMING` (i.e. bit distance)
- `crossCheck` - `False` by default; if `True` only returns matches which are "symmetrical", i.e. where a pair of descriptors are each others best matches

The `cv2.BFMatcher.match(queryDescr, trainDescr)` method returns a `list` of `DMatch` objects:
- `DMatch.distance` - distance between descriptors using `normType`
- `DMatch.queryIdx` - index of the descriptor in query descriptors
- `DMatch.trainIdx` - index of the descriptor in train descriptors

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(des1,des2)
matches = sorted(matches, key=lambda x: x.distance)

In [ ]:
im_match = cv2.drawMatches(im1,kp1,im2,kp2,matches[:65],None,matchColor=(0,255,0),flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
show_img(im_match, figsize=(30,30));

## FLANN (Fast Library for Approximate Nearest Neighbors) matching

In [ ]:
FLANN_INDEX_KDTREE=1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)

matches = flann.match(des1,des2)
matches = sorted(matches, key=lambda x: x.distance)

In [ ]:
im_match = cv2.drawMatches(im1,kp1,im2,kp2,matches[:5],None,matchColor=(0,255,0),flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
show_img(im_match, figsize=(30,30));

## Homography

In [ ]:
im1 = cv2.imread('Data/student201.jpeg')
im2 = cv2.imread('Data/s103.jpg')
show_imgs((im1,im2), rows=1, figsize=(24,12));

In [ ]:
descr = cv2.ORB_create()
kp1,des1 = descr.detectAndCompute(im1, None)
kp2,des2 = descr.detectAndCompute(im2, None)

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)
matches = sorted(matches, key=lambda x: x.distance)

In [ ]:
im_match = cv2.drawMatches(im1,kp1,im2,kp2,matches[:25],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
show_img(im_match, figsize=(16,16));